In [ ]:
#TO IMPORT YOUR KAGGLE DATA SOURCES,

import kagglehub
arshkon_linkedin_job_postings_path = kagglehub.dataset_download('arshkon/linkedin-job-postings')

print('Data source import complete.')


100%|██████████| 159M/159M [00:01<00:00, 141MB/s]

Extracting files...


Data source import complete.


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_job_posting = pd.read_csv('/content/postings.csv')
df_companies = pd.read_csv('/content/companies.csv')
df_comp_industries = pd.read_csv('/content/company_industries.csv')
df_comp_specialties = pd.read_csv('/content/company_specialities.csv')

In [ ]:
print(df_job_posting.dtypes)
df_job_posting.shape

job_id                          int64
company_name                   object
title                          object
description                    object
max_salary                    float64
pay_period                     object
location                       object
company_id                    float64
views                         float64
med_salary                    float64
min_salary                    float64
formatted_work_type            object
applies                       float64
original_listed_time          float64
remote_allowed                float64
job_posting_url                object
application_url                object
application_type               object
expiry                        float64
closed_time                   float64
formatted_experience_level     object
skills_desc                    object
listed_time                   float64
posting_domain                 object
sponsored                       int64
work_type                      object
currency    

(123849, 31)

In [ ]:
print(df_companies.dtypes)
df_companies.shape

company_id        int64
name             object
description      object
company_size    float64
state            object
country          object
city             object
zip_code         object
address          object
url              object
dtype: object


(24473, 10)

In [ ]:
print(df_comp_industries.dtypes)
df_comp_industries.shape

company_id     int64
industry      object
dtype: object


(24375, 2)

In [ ]:
print(df_comp_specialties.dtypes)
df_comp_specialties.shape

company_id     int64
speciality    object
dtype: object


(169387, 2)

In [ ]:
#Count of blanks / missing values per table

# Count blanks (NaNs) per attribute
blank_counts_df_posting = df_job_posting.isna().sum()
blank_counts_df_companies = df_companies.isna().sum()
blank_counts_df_comp_industries = df_comp_industries.isna().sum()
blank_counts_df_comp_specialties = df_comp_specialties.isna().sum()

print("Count of blanks per attribute in job_posting:")
print(blank_counts_df_posting)

print("Count of blanks per attribute in job_companies:")
print(blank_counts_df_companies)

print("Count of blanks per attribute in comp_industries:")
print(blank_counts_df_comp_industries)

print("Count of blanks per attribute in comp_specialties:")
print(blank_counts_df_comp_specialties)

Count of blanks per attribute in job_posting:
job_id                             0
company_name                    1719
title                              0
description                        7
max_salary                     94056
pay_period                     87776
location                           0
company_id                      1717
views                           1689
med_salary                    117569
min_salary                     94056
formatted_work_type                0
applies                       100529
original_listed_time               0
remote_allowed                108603
job_posting_url                    0
application_url                36665
application_type                   0
expiry                             0
closed_time                   122776
formatted_experience_level     29409
skills_desc                   121410
listed_time                        0
posting_domain                 39968
sponsored                          0
work_type                    

In [ ]:
#Drop rows where company_name is blank (NaN)
df_cleaned_comp_name = df_job_posting.dropna(subset=['company_name'])

In [ ]:
#Drop rows where views is blank (NaN)
df_cleaned_views = df_cleaned_comp_name.dropna(subset=['views'])
df_cleaned_views.shape

(120503, 31)

In [ ]:
#Rename 'description' attributes to identify between job and company description
df_cleaned_views.rename(columns={'description': 'job_description'}, inplace=True)
df_companies.rename(columns={'description': 'comp_description'}, inplace=True)

#Join df_cleaned_views, df_companies, df_comp_industries, df_comp_specialties

#Left Join
left_join = pd.merge(df_cleaned_views[['company_id','company_name','title','job_description','formatted_work_type','skills_desc','location','views']],
                     df_companies[['company_id','comp_description']],
                     on='company_id', how='left')
left_join.shape

<ipython-input-15-081db310395c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned_views.rename(columns={'description': 'job_description'}, inplace=True)


(120503, 9)

In [ ]:
#Left Join
left_join = pd.merge(left_join,
                     df_comp_industries[['company_id','industry']],
                     on='company_id', how='left')
left_join.shape

(120650, 10)

In [ ]:
#Normalize comp_specialties table
aggregated_specialties = df_comp_specialties.groupby('company_id')['speciality'].apply(lambda x: ' '.join(x)).reset_index()

aggregated_specialties.shape
aggregated_specialties.head(3)

,company_id,speciality
0,1009,Cloud Mobile Cognitive Security Research Watso...
1,1016,Healthcare Biotechnology
2,1028,enterprise software applications database midd...


In [ ]:
#Final Left Join
joined_data = pd.merge(left_join,
                     aggregated_specialties[['company_id','speciality']],
                     on='company_id', how='left')

joined_data.shape

(120650, 11)

In [ ]:
#Check blank values in final dataset
blank_joined_data = joined_data.isna().sum()
print("Count of blanks per attribute in job_posting:")
print(blank_joined_data)

Count of blanks per attribute in job_posting:
company_id                  0
company_name                0
title                       0
job_description             6
formatted_work_type         0
skills_desc            118285
location                    0
views                       0
comp_description         1005
industry                  153
speciality              25764
dtype: int64


In [ ]:
#Create new feature combining title, job_description, skills_desc
joined_data['job_details'] = joined_data['title'] + ' ' + joined_data['job_description'] + ' ' + joined_data['skills_desc']


joined_data.shape

(120650, 12)

In [ ]:
#Check blank values in final dataset
blank_joined_data = joined_data.isna().sum()
print("Count of blanks per attribute in job_posting:")
print(blank_joined_data)

Count of blanks per attribute in job_posting:
company_id                  0
company_name                0
title                       0
job_description             6
formatted_work_type         0
skills_desc            118285
location                    0
views                       0
comp_description         1005
industry                  153
speciality              25764
job_details            118285
dtype: int64


In [ ]:
#Replace NaN
joined_data['title'] = joined_data['title'].fillna(' ')
joined_data['job_description'] = joined_data['job_description'].fillna(' ')
joined_data['skills_desc'] = joined_data['skills_desc'].fillna(' ')

joined_data['comp_description'] = joined_data['comp_description'].fillna('missing company description')
joined_data['speciality'] = joined_data['speciality'].fillna('missing speciality')
joined_data['industry'] = joined_data['industry'].fillna('missing industry')

In [ ]:
joined_data['job_details'] = joined_data['title'] + ' ' + joined_data['job_description'] + ' ' + joined_data['skills_desc']

In [ ]:
blank_joined_data = joined_data.isna().sum()
print("Count of blanks per attribute in job_posting:")
print(blank_joined_data)

Count of blanks per attribute in job_posting:
company_id             0
company_name           0
title                  0
job_description        0
formatted_work_type    0
skills_desc            0
location               0
views                  0
comp_description       0
industry               0
speciality             0
job_details            0
dtype: int64


In [ ]:
#Check missing job_details
nan_records = joined_data[joined_data['job_details'].isna()]
print(nan_records)

Empty DataFrame
Columns: [company_id, company_name, title, job_description, formatted_work_type, skills_desc, location, views, comp_description, industry, speciality, job_details]
Index: []


In [ ]:
# Referential integrity
# Filter out job postings with invalid company_ids
valid_company_ids = df_companies['company_id'].unique()  # Get unique company_ids from companies table
df_job_posting = df_job_posting[df_job_posting['company_id'].isin(valid_company_ids)]

# Now the following assertion should pass:
assert df_job_posting['company_id'].isin(df_companies['company_id']).all(), "Referential integrity failed for company_id in job postings."

# Check for company_id consistency in industries and specialties tables
assert df_comp_industries['company_id'].isin(df_companies['company_id']).all(), "Referential integrity issue in industries table."
assert df_comp_specialties['company_id'].isin(df_companies['company_id']).all(), "Referential integrity issue in specialties table."


In [ ]:
# Field-level integrity
# Validate views are non-negative and realistic
assert df_job_posting['views'].dropna().apply(lambda x: x >= 0 and x <= 100000).all(), "Invalid values in 'views'."

# Get unique industries from the DataFrame
valid_industries = df_comp_industries['industry'].unique().tolist()

# Check for valid industries (now using all unique industries)
assert df_comp_industries['industry'].isin(valid_industries).all(), "Invalid industries found."
